In [20]:
# Se importan las bibliotecas necesarias
import numpy as np
import pandas as pd
import re
import warnings
warnings.filterwarnings('ignore')

import plotly.plotly as py
import plotly.graph_objs as go

#nwH7euTNo27AATjelWEi

In [22]:
date_parser = pd.to_datetime

In [23]:
df = pd.read_csv('/home/pradel/Desktop/aire_cdmx/apps/dataset/datos_pau.csv'
                 , parse_dates=['fecha'], error_bad_lines=False)

In [24]:
df = df.sort_values(['fecha'], ascending=[0])

In [25]:
df.dtypes

fecha                  datetime64[ns]
O3_pronóstico                 float64
PM10_pronóstico               float64
O3_y_frcst_12                 float64
PM10mean_y_frcst_12           float64
TMP_x                         float64
TMP_y                         float64
O3_y                          float64
O3                            float64
O3_x                          float64
RH                            float64
TMP                           float64
PM10                          float64
PM10_y                        float64
PM10_x                        float64
PM10mean_y                    float64
dtype: object

In [4]:
def convertir_ppb(row, columna):
    ppb = row[columna]/1.96
    return ppb

In [5]:
def convertir_unidades(df, columna_fecha):
    df = df.sort_values([columna_fecha], ascending=[0])
    df['PM10mean_y_ppb'] = df.apply(lambda row: convertir_ppb(row, 'PM10mean_y'), axis=1)
    df_concentraciones = df[['fecha', 'O3_y','PM10mean_y_ppb', 'TMP', 'TMP_x', 'TMP_y']]

    return df_concentraciones

In [6]:
df_concentraciones = convertir_unidades(df, 'fecha')

In [7]:
df_concentraciones.head()

,fecha,O3_y,PM10mean_y_ppb,TMP,TMP_x,TMP_y
23,2019-06-26 17:00:00,NaN,NaN,NaN,NaN,NaN
22,2019-06-26 16:00:00,NaN,NaN,NaN,NaN,NaN
21,2019-06-26 15:00:00,NaN,NaN,NaN,NaN,NaN
20,2019-06-26 14:00:00,NaN,NaN,NaN,NaN,NaN
19,2019-06-26 13:00:00,NaN,NaN,NaN,NaN,NaN


In [8]:
dic_colores = {'O3_pronóstico': '#d73027'
                   , 'PM10_pronóstico': '#9970ab'
                   , 'O3_y_frcst_12': '#313695'
                   , 'PM10mean_y_frcst_12': '#40004b'
                   , 'TMP_x': '#80cdc1'
                   , 'TMP_y': '#003c30'
                   , 'O3_y': '#a50026'
                   , 'O3': '#f46d43'
                   , 'O3_x': '#fdae61'
                   , 'RH': '#c51b7d'
                   , 'TMP': '#35978f'
                   , 'PM10': '#542788'
                   , 'PM10_y': '#9970ab'
                   , 'PM10_x': '#c2a5cf'
                   , 'PM10mean_y': '#762a83'
                   , 'PM10mean_y_ppb': '#762a83'
                   , 'fecha': '#d73027'}

In [9]:
dic_etiquetas = {'O3_pronóstico': 'Pronóstico de O3'
                     , 'PM10_pronóstico': 'Pronóstico de PM10'
                     , 'O3_y_frcst_12': 'Pronóstico histórico de O3'
                     , 'PM10mean_y_frcst_12': 'Pronóstico histórico de PM10'
                     , 'TMP_x': 'Temperatura mínima'
                     , 'TMP_y': 'Temperatura máxima'
                     , 'O3_y': 'O3 máximo'
                     , 'O3': 'O3 promedio'
                     , 'O3_x': 'O3 mínimo'
                     , 'RH': 'Humedad relativa'
                     , 'TMP': 'Temperatura promedio'
                     , 'PM10': 'PM10 promedio'
                     , 'PM10_ppb': 'PM10 promedio'
                     , 'PM10_y': 'PM10 máximo'
                     , 'PM10_y_ppb': 'PM10 máximo'
                     , 'PM10_x': 'PM10 mínimo'
                     , 'PM10_x_ppb': 'PM10 mínimo'
                     , 'PM10mean_y': 'PM10 promedio 24 hrs'
                     , 'PM10mean_y_ppb': 'PM10 máximo (ppb)'
                     , 'fecha': 'fecha'}

In [10]:
def crear_trazo(df, y, name, color, x='fecha', width=2):
    regex = re.findall('pronóstico', name)
    if regex:
        trazo = go.Scatter(x=df[x],
                           y=df[y],
                           name=name,
                           line=dict(color=color, width=width, dash='dot'))
    else:
        trazo = go.Scatter(x=df[x],
                           y=df[y],
                           name=name,
                           line=dict(color=color, width=width))
    return trazo

In [11]:
df_filtro=df_concentraciones[(df_concentraciones["fecha"]>"2019-06-26 06:00:00") & (df_concentraciones["fecha"]<"2019-06-26 16:00:00")]

In [12]:
df_filtro.head()

,fecha,O3_y,PM10mean_y_ppb,TMP,TMP_x,TMP_y
21,2019-06-26 15:00:00,NaN,NaN,NaN,NaN,NaN
20,2019-06-26 14:00:00,NaN,NaN,NaN,NaN,NaN
19,2019-06-26 13:00:00,NaN,NaN,NaN,NaN,NaN
18,2019-06-26 12:00:00,NaN,NaN,NaN,NaN,NaN
17,2019-06-26 11:00:00,NaN,NaN,NaN,NaN,NaN


In [17]:
df_filtro.dtypes

fecha             datetime64[ns]
O3_y                     float64
PM10mean_y_ppb           float64
TMP                      float64
TMP_x                    float64
TMP_y                    float64
dtype: object

In [14]:
#df_filtro.columns

In [15]:
data = [crear_trazo(df_concentraciones, col, dic_etiquetas[col], dic_colores[col]) for col in df_filtro.columns[1:]]

In [16]:
layout = dict (title='Pronóstico',
               xaxis={'title': 'fecha'},
               yaxis={'title': 'ppm (partículas por millón)'})


fig = dict(data=data, layout=layout)
py.iplot(fig, filename='styled-line')